In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
from sklearn.metrics import classification_report

https://medium.com/stanford-cs224w/graph-neural-networks-for-knowledge-tracing-ef31fdaa5f00

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df=pd.read_pickle('appmath.pkl')  

In [6]:
df2=pd.read_csv('prereq_edges.csv')

In [7]:
G=nx.Graph()

for _,i in df2.iterrows():    
       
        G.add_edge(i['source_lo_title'],i['dest_lo_title'])
        
subgraphs=[G.subgraph(i) for i in nx.connected_components(G)]

for num,sub in enumerate(subgraphs):
    
    nx.set_node_attributes(sub,num,'subGraphId')

nodeSubGraphId={}
for g in subgraphs:
    for node in g.nodes(data=True):
         nodeSubGraphId[node[0]]= node[1]['subGraphId']
            
subgraphsNodeCountDict={ num: g.number_of_nodes() for num,g in  enumerate(subgraphs)}

In [8]:
for col in df.columns:
    if df[col].dtype.name=='category':
        df[col]=df[col].astype(str)

In [9]:
df['loGraphID']=df.learning_objective_name.apply(lambda x: nodeSubGraphId.get(x) )

In [10]:
difficulty=df[~df.correct.isna()].groupby('atom_id')['correct'].mean().reset_index().rename(columns={'correct':'difficulty'})

In [11]:
difficulty

,atom_id,difficulty
0,000361fc-3126-4f25-93da-3ee3b66021c4,0.571429
1,000785e5-46df-4d0b-a12d-2283db4b8c7f,1.000000
2,000da431-9e35-4309-a3ae-cca9225b8d54,0.638554
3,00170926-4ba4-4d1e-bd86-0ac94107acb1,0.700000
4,001cfd36-68fa-4d74-b077-769f3bf2e0ad,0.712919
...,...,...
16441,fff0e6fc-e78f-41da-bcc8-88a6dceb6cc3,0.581481
16442,fff4801c-d828-453e-b1d4-64369f7b1048,0.666667
16443,fff652bc-b348-4054-9aab-8e67b98b022a,0.714286
16444,fffbe876-95cc-4dd0-9097-c657f446df6a,0.714286


In [12]:
df=df.merge(difficulty,how='left',on='atom_id')

In [13]:
loIndex={lo:i for i,lo in enumerate( df.learning_objective_name.unique())}
df['loIndex']=df['learning_objective_name'].apply(lambda x: loIndex[x])


#df2['source_index']=df2.source_lo_title.map(loIndex)
#df2['dest_index']=df2.dest_lo_title.map(loIndex)

#df2=df2[['source_index','dest_index']].reset_index(drop=True)
#loEdgeIndex=loEdgeMapping.values.transpose()

In [14]:
#df2[(df2.source_lo_title.isin( df.learning_objective_name.unique()[:5])) | (df2.dest_lo_id.isin( df.learning_objective_name.unique()[:5]))]

In [15]:
#loEdgeMapping=df2[['source_index','dest_index']].reset_index(drop=True)
#loEdgeMapping=loEdgeMapping.dropna(axis=0)
#loEdgeIndex=loEdgeMapping.values.transpose()

In [16]:
#loEdgeIndex

In [17]:
#atomIndex={atomId:num for num,atomId in enumerate( df.atom_id.to_list())}

In [18]:
#df['atomIndex']=df.atom_id.apply(lambda x : atomIndex[x] )

In [19]:
def atomClassifier(x):
    if x==None:
        return 'learningMaterial'
    else:
        return 'question'

In [20]:
df['atom_type']=df.correct.apply(lambda x:atomClassifier(x) )

In [21]:
df['correctBinary']=df.correct*1
df.atom_id=df.atom_id.astype(str)


In [22]:
def user_dataset(df,df2,user,goal):
    
    '''
    df - df should be filtered by goal id
    
    edges
    -------
    lo -> atom
    atom -> atom
    lo -> learning materials (lm)
    atom -> lm
    lm -> atom
    lm -> lm
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty , number of attempt
    lm - time spend 
        
    '''    
    df=df[(df.loGraphID==goal) & (df.user_id==user )   ].reset_index(drop=True)
    userDf=df.sort_values(by='interaction_end_time')
    loIndex={lo:i for i,lo in enumerate( userDf.learning_objective_name.unique())}
    
    
    userDf['loIndex']=userDf['learning_objective_name'].apply(lambda x: loIndex[x])
    
    unique_lo=df2[(df2.source_lo_title.isin( df.learning_objective_name.unique())) | (df2.dest_lo_id.isin( df.learning_objective_name.unique()))]
    
    unique_lo['source_index']=unique_lo.source_lo_title.map(loIndex)
    unique_lo['dest_index']=unique_lo.dest_lo_title.map(loIndex)
    
    loEdgeMapping=unique_lo[['source_index','dest_index']].reset_index(drop=True)
    loEdgeMapping=loEdgeMapping.dropna(axis=0)
    loEdgeIndex=loEdgeMapping.values.transpose()
    
    #userDf=df[df.user_id==user]    
    
    
    #atomIndex={atomId:num for num,atomId in enumerate( userDf[userDf.atom_type=='question'].atom_id.to_list())}
    #lmIndex={atomId:num for num,atomId in enumerate( userDf[userDf.atom_type=='learningMaterial'].atom_id.to_list())}
    #
    ## assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    #userDf.loc[userDf.atom_type=='question','atomIndex']=userDf.loc[userDf.atom_type=='question',:].atom_id.apply(lambda x : atomIndex[x] )
    #userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=userDf.loc[userDf.atom_type=='learningMaterial',:].atom_id.apply(lambda x : lmIndex[x] )
    
    
    atomIndex=np.arange(userDf[userDf.atom_type=='question'].atom_id.shape[0])
    lmIndex=np.arange(userDf[userDf.atom_type=='learningMaterial'].atom_id.shape[0])
    
    # assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    userDf.loc[userDf.atom_type=='question','atomIndex']=atomIndex
    userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=lmIndex
    
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    
    userDf['attempt_count']=userDf.groupby(by=['atom_id']).cumcount()+1
    
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    #atom features
    atomFeatures = userDf[userDf.atom_type=='question'][['difficulty']].to_numpy()
    #learning materials features
    lmFeatures = userDf[userDf.atom_type=='learningMaterial'][['duration_s']].to_numpy()
    #learning objectives
    loFeatures=np.ones(userDf.loIndex.nunique()).reshape(-1,1)
    #target
    target=userDf[userDf.atom_type=='question']['correctBinary'].astype(int).to_numpy()
    
    userDf_shift= userDf[~userDf.atomIndex_shift.isna()].copy()  
    
    #atom-atom
    atomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #atom-lm
    atomLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-atom
    LmAtomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-lm
    LmLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    featuresDict={'atom':atomFeatures,'lm':lmFeatures,'lo':loFeatures}
    edgesDict={'lo_atom':loAtomEdgeIndex,'lo_lm':loLmEdgeIndex,'lo':loEdgeIndex,'atom': atomEdgeIndex,'atom_lm':atomLmEdgeIndex,'lm_atom':LmAtomEdgeIndex,'lm_lm':LmLmEdgeIndex}
    
    return featuresDict,edgesDict,target
    
    
    
    
    

df[['graph_id', 'user_id', 
       'goal_id',
        'interaction_end_time',
       'learning_objective_name', 'atom_id', 
       'correct', 'time_spent_answering_s', 'time_spent_on_instruction_s',
       'goal_progress',
       'target_status_and_progress', 'prev_concept_narrative',
       'duration_s', 'is_target', 'loGraphID', 'difficulty', 'loIndex',
       'atom_type', 'correctBinary']].to_pickle('EDA_data.pickle')

In [23]:
df.head()

,graph_id,course_id,section_id,user_id,registration_id,goal_id,path_id,path_type,adaptive_behavior,topic_id,lo_to_concept_blacklist,interaction_end_time,concept_id,concept_partner_id,concept_name,retail_lo_id,learning_objective_name,atom_id,retail_atom_id,variation_id,sequence_created_at,parent_sequence_id,sequence_instance_id,sequence_ordinal,recommendation_id,source,state,type,correct,time_spent_answering_s,time_spent_on_instruction_s,goal_progress,work_remaining,status,target_status_and_progress,prev_concept_narrative,current_concept_narrative,focused_concept_id,prev_lo_narrative,current_lo_narrative,focused_lo_id,focused_topic_id,duration_s,is_target,loGraphID,difficulty,loIndex,atom_type,correctBinary
0,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,0bbde3de-283b-4b08-a783-1a2942d07a2e,e359efc4-ed49-4227-a03e-c0c9358487d3,009f012a-30c9-42c5-94c6-3e05008ccfd0,REVIEW,None,d5743794-1f74-46ca-a538-8bf6c1cbb30b,NaN,2022-10-02 16:17:07.342,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,90bf68ef-fa40-4c57-9054-c417a848d897,5e457b03-88e7-4798-9c17-0c44a1c2d419,4ffcf121-73c3-4e21-adb9-d117faecffe9,2022-10-02 15:57:10.382,5baf1afa-5bc6-4c72-a7ac-b1bd4f180d22,c3b5bff2-e288-473d-bc1b-88c38be34d35,1,6.982368e+18,RECOMMENDATION,DONE,ASSESS,False,396.441010,24.492001,0.037663,-1.0,in_progress,{'53568360-e9fe-465f-a8fc-1c75027cba94': {'pro...,DIRECT_CHOOSE_TOPIC,DIRECT_DIAGNOSE_TOPIC,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,NONE,NONE,ff1bfcee-603f-4b08-924a-3852972fffab,tref-d5743794-1f74-46ca-a538-8bf6c1cbb30b,420.933014,True,3.0,0.525000,0,question,0
1,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,0bbde3de-283b-4b08-a783-1a2942d07a2e,e359efc4-ed49-4227-a03e-c0c9358487d3,009f012a-30c9-42c5-94c6-3e05008ccfd0,REVIEW,None,d5743794-1f74-46ca-a538-8bf6c1cbb30b,NaN,2022-10-02 16:17:42.073,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,49287c70-b63d-41be-9acc-24fd1f273158,614cad88-03e6-4c22-9689-aab68b3f23ba,5dd2e89c-bbcf-41ca-a91c-052010465d40,2022-10-02 16:17:07.893,a9af40ef-a988-4a30-b1fc-2344676d4cc7,408fc057-0815-4940-8ede-1ebe8207c1f0,1,6.982373e+18,RECOMMENDATION,DONE,ASSESS,True,26.042000,8.122000,0.533186,-1.0,in_progress,{'53568360-e9fe-465f-a8fc-1c75027cba94': {'pro...,DIRECT_DIAGNOSE_TOPIC,DIRECT_DIAGNOSE_TOPIC,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,NONE,NONE,ff1bfcee-603f-4b08-924a-3852972fffab,tref-d5743794-1f74-46ca-a538-8bf6c1cbb30b,34.164001,True,3.0,0.547619,0,question,1
2,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,0bbde3de-283b-4b08-a783-1a2942d07a2e,e359efc4-ed49-4227-a03e-c0c9358487d3,009f012a-30c9-42c5-94c6-3e05008ccfd0,REVIEW,None,d5743794-1f74-46ca-a538-8bf6c1cbb30b,NaN,2022-10-02 16:17:42.341,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,b2b1e778-26e0-4c99-ad3f-8ac5e8838a49,fa2a4b85-28c2-460d-b6ee-3e4686ad7db8,246cb150-9245-4042-b9b3-9392bab4ccfc,2022-10-02 16:17:42.341,340e10ca-1ab2-4559-8e89-4cd3eac53277,a3dbdfe0-2d1f-4b6b-927c-366acdbc0763,1,6.982373e+18,RECOMMENDATION,SHOWN,ASSESS,None,0.000000,0.000000,NaN,NaN,nan,NaN,REVIEW_SWITCH,REVIEW_PRE_TEST,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,NONE,NONE,ff1bfcee-603f-4b08-924a-3852972fffab,tref-d5743794-1f74-46ca-a538-8bf6c1cbb30b,0.000000,True,3.0,0.433962,0,learningMaterial,NaN
3,076e1a0e-669

In [24]:
df[(df.loGraphID==3)].user_id.unique()[:5]

array(['0012ff2b-871a-45d6-8e33-804f7464fa0a',
       '024c3766-49e8-4265-9da7-6853757a6a41',
       '025fda22-a146-476d-9a8d-d6bd3052c181',
       '038f2ad6-cdc2-40de-aacb-4a62c131598c',
       '03cff449-ff27-4542-83fe-d676dc64eaaf'], dtype=object)

In [25]:
features, edges,target =user_dataset(df,df2,'0012ff2b-871a-45d6-8e33-804f7464fa0a',3 )

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_19972\1155036963.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['source_index']=unique_lo.source_lo_title.map(loIndex)
C:\Users\mpathirana\AppData\Local\Temp\ipykernel_19972\1155036963.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['dest_index']=unique_lo.dest_lo_title.map(loIndex)


In [26]:
for key,item in features.items():
    print( '{} : {}'.format(key,item.shape))

atom : (293, 1)
lm : (54, 1)
lo : (27, 1)


In [27]:
for key,item in edges.items():
    print( '{} : {}'.format(key,item.shape))

lo_atom : (2, 293)
lo_lm : (2, 54)
lo : (2, 35)
atom : (2, 244)
atom_lm : (2, 49)
lm_atom : (2, 48)
lm_lm : (2, 5)


In [28]:
from torch_geometric.data import HeteroData
import torch 


In [29]:
import torch_geometric.transforms as T

In [30]:
def generate_data(features,edges,y):  
    data = HeteroData()
    #node features
    data['atom'].x=torch.tensor(features['atom']).float()
    data['lo'].x= torch.tensor(features['lo']).float()
    data['lm'].x= torch.tensor(features['lm']).float()
    
    #edge index
    data['lo','lo_atom','atom'].edge_index=torch.from_numpy(edges['lo_atom']).long()
    data['lo','lo_lo','lo'].edge_index= torch.from_numpy(edges['lo']).long()
    data['atom','atom_atom','atom'].edge_index=torch.from_numpy(edges['atom']).long()
    data['lo','lo_lm','lm'].edge_index=torch.from_numpy(edges['lo_lm']).long()
    data['atom','atom_lm','lm'].edge_index=torch.from_numpy(edges['atom_lm']).long()
    data['lm','atom_lm','atom'].edge_index=torch.from_numpy(edges['lm_atom']).long()
    data['lm','lm_lm','lm'].edge_index=torch.from_numpy(edges['lm_lm']).long()

    data['atom'].y=torch.from_numpy(y).long()


    #split=T.RandomNodeSplit(num_test =5)
    #data=split(data)

    size=y.shape[0]
    #num_test_nodes =int(y.shape[0]*.001)
    #some_indices = torch.randperm(size)[:num_test_nodes]
    some_indices = torch.randperm(size)[0]


    test_mask = torch.zeros(y.shape[0], dtype=torch.bool)
    test_mask[some_indices] = True
    data['atom'].test_mask = test_mask

    train_mask = torch.ones(y.shape[0], dtype=torch.bool)
    train_mask[some_indices] = False
    data['atom'].train_mask = train_mask

    normalize=T.NormalizeFeatures()
    data=normalize(data)
    return data

In [31]:
data=generate_data(features, edges,target)

In [32]:
users=df[(df.loGraphID==3)].user_id.unique()

In [34]:
#df[(df.loIndex==3) ].to_csv('loindex_3_analysis_data.csv')

In [35]:
users.shape

(326,)

In [36]:
usersGraphDataList=[]
for user in users: 
    try:
        features, edges,target =user_dataset(df,df2,user,3 )
        data=generate_data(features, edges,target)
        usersGraphDataList.append(data)
    except:
        pass
    

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_19972\1155036963.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['source_index']=unique_lo.source_lo_title.map(loIndex)
C:\Users\mpathirana\AppData\Local\Temp\ipykernel_19972\1155036963.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['dest_index']=unique_lo.dest_lo_title.map(loIndex)
C:\Users\mpathirana\AppData\Local\Temp\ipykernel_19972\1155036963.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [37]:
print('data.has_isolated_nod : {}'.format(data.has_isolated_nodes()))
print('data.has_self_loops   : {}'.format(data.has_self_loops()))
print('data.is_undirected    : {}'.format(data.is_directed()))

data.has_isolated_nod : False
data.has_self_loops   : False
data.is_undirected    : True


In [38]:
import torch_geometric
from torch_geometric.nn import GCNConv 
from torch_geometric.nn import SAGEConv, to_hetero ,LSTMAggregation
import  torch.nn as nn
import torch.nn.functional as F
import torch 
from torch_geometric.loader import DataLoader

models : https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html#:~:text=Creating%20Heterogeneous%20Graphs&text=Node%20types%20are%20identified%20by,the%20edge%20type%20can%20exist.

In [60]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn1=nn.ReLU()
        
        self.conv2 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn2=nn.ReLU()
        
        self.conv3 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn3=nn.ReLU()
        
        #self.linear=nn.Linear(hidden_channels,hidden_channels)
        
        self.lstm = nn.LSTM(input_size=hidden_channels, hidden_size=hidden_channels/2)
        
        self.out=nn.Linear(hidden_channels/2,out_channels)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x, edge_index):
        # First Message Passing layer
        x = self.actfn1(self.conv1(x, edge_index))
        x= F.dropout(x,p=.2,training=self.training)
        
        # Second Message Passing layer
        x = self.actfn2(self.conv2(x, edge_index))
        x= F.dropout(x,p=.2,training=self.training)
        
        # Third Message Passing layer  
        #x = self.actfn3(self.conv3(x, edge_index))
        #x= F.dropout(x,p=.25,training=self.training)
        #x = self.linear(x)
        x, _ = self.lstm(x)
       
        
        # Out layer
        x=self.sigmoid( self.out(x))
        
        return x


model_1 = GNN(hidden_channels=64, out_channels=2)
model_1 = to_hetero(model_1, data.metadata(), aggr='sum')

In [40]:
import torch.nn as nn

class GNN_LSTM(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = SAGEConv((-1, -1), hidden_channels, requires_grad=False)
        self.actfn1 = nn.PReLU()

        self.lstm = nn.LSTMCell(hidden_channels, hidden_channels)

        self.conv2 = SAGEConv((-1, -1), hidden_channels, requires_grad=False)
        self.actfn2 = nn.PReLU()

        self.out = nn.Linear(hidden_channels, out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index):
        # First Message Passing layer
        x = self.actfn1(self.conv1(x, edge_index))
        x = F.dropout(x, p=.25, training=self.training)

        # LSTM layer
        hx = torch.zeros(x.size(0), self.lstm.hidden_size, device=x.device)
        cx = torch.zeros(x.size(0), self.lstm.hidden_size, device=x.device)

        for _ in range(3):
            hx, cx = self.lstm(x, hx, cx)

        x = hx

        # Second Message Passing layer
        x = self.actfn2(self.conv2(x, edge_index))
        x = F.dropout(x, p=.25, training=self.training)

        # Out layer
        x = self.sigmoid(self.out(x))

        return x




model_3 = GNN_LSTM(hidden_channels=64, out_channels=2)
model_3 = to_hetero(model_1, data.metadata(), aggr='sum')

C:\Users\mpathirana\Anaconda3\lib\site-packages\torch_geometric\nn\to_hetero_transformer.py:379: UserWarning: 'actfn1.atom' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'actfn1.atom'
  warnings.warn(
C:\Users\mpathirana\Anaconda3\lib\site-packages\torch_geometric\nn\to_hetero_transformer.py:379: UserWarning: 'actfn1.lo' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'actfn1.lo'
  warnings.warn(
C:\Users\mpathirana\Anaconda3\lib\site-packages\torch_geometric\nn\to_hetero_transformer.py:379: UserWarning: 'actfn1.lm' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'actfn1.lm'
  warnings.warn(
C:\Users\mpathirana\Anaconda3\lib\site-packages\torch_geometric\nn\to_hetero_transformer.py:379: UserWarning: 'actfn2.atom' will be duplicated, but its parameters cannot be reset. To sup

In [41]:
from torch_geometric.nn import GATConv, Linear, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        
        
        
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        
        x = self.conv2(x, edge_index) + self.lin2(x)
        x=x.relu()
        
        return x

#
#model_2 = GAT(hidden_channels=64, out_channels=2)
#model_2 = to_hetero(model_2, data.metadata(), aggr='mean')

model=model_1
dataLoader=list(DataLoader(usersGraphDataList,batch_size=80))
learning_rate=0.01
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=torch.nn.CrossEntropyLoss()
train_acc_mean_list=[]
test_acc_mean_list=[]
loss_list=[]
for epcoh in range(30):
    train_acc_list=[]
    test_acc_list=[]
    for num, data in enumerate(dataLoader):
        
        model.train()
        optimizer.zero_grad()
        #data.x = torch.from_numpy(data.x_dict)
        #data.edge_index = torch.from_numpy(data.edge_index_dict)
        out=model(data.x_dict, data.edge_index_dict)  # model1
        #out = model(data.x, data.edge_index)
        #BCELoss -> loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
        #loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
        loss=criterion(out['atom'][data['atom'].train_mask],data['atom'].y[data['atom'].train_mask])
        loss.backward()
        
        optimizer.step()
        pred=out['atom'].argmax(dim=1)
        train_correct=pred[data['atom'].train_mask]==data['atom'].y[data['atom'].train_mask]
        train_acc=int(train_correct.sum())/int(data['atom'].train_mask.sum())
        model.eval()
        
        pred=out['atom'].argmax(dim=1)
        test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
        test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        loss_list.append(loss.item())
    train_acc_mean_list.append(np.mean(train_acc_list))
    test_acc_mean_list.append(np.mean(test_acc_list))
    
        
    print('Epoch : {} ,Train Batch : {} , loss : {} , accuracy : {:.4f} '.format(epcoh+1,num+1,loss,np.mean(train_acc_list)))
    print('Epoch : {} ,Train Batch : {} , loss : {} , accuracy : {:.4f} \n'.format(epcoh+1,num+1,loss,np.mean(test_acc_list)))

fig,axs=plt.subplots(1,2,figsize=(12,7))
axs[1].plot(train_acc_mean_list,label='train')
axs[1].plot(test_acc_mean_list,label='test')
axs[0].plot(loss_list,label='loss')
plt.legend();
plt.grid()

In [42]:
usersGraphDataList
def batch_generator(usersGraphDataList):
    
    np.random.shuffle(usersGraphDataList)
    test_size=int(len(usersGraphDataList)*.2)
    test_data=usersGraphDataList[:test_size]
    train_data=usersGraphDataList[test_size:]
    
    return test_data,train_data
    

In [ ]:
model_1 = GNN(hidden_channels=200, out_channels=2)
model_1 = to_hetero(model_1, data.metadata(), aggr='mean')


model_2 = GAT(hidden_channels=100, out_channels=2)
model_2 = to_hetero(model_2, data.metadata(), aggr='mean')


model=model_1
#dataLoader=list(DataLoader(usersGraphDataList,batch_size=80))
test_data,train_data=batch_generator(usersGraphDataList)
learning_rate=0.01
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=torch.nn.CrossEntropyLoss()
train_acc_mean_list=[]
test_acc_mean_list=[]
loss_list=[]
best_score=0
for epcoh in range(10):
    train_acc_list=[]
    test_acc_list=[]
    loss_list=[]
    for num, data in enumerate(train_data):
        
        model.train()
        optimizer.zero_grad()
        #data.x = torch.from_numpy(data.x_dict)
        #data.edge_index = torch.from_numpy(data.edge_index_dict)
        out=model(data.x_dict, data.edge_index_dict)  # model1
        #out = model(data.x, data.edge_index)
        #BCELoss -> loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
        #loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
        loss=criterion(out['atom'],data['atom'].y)
        #loss=criterion(out['atom'].argmax(dim=1).float().requires_grad_(True),data['atom'].y.float())
        loss.backward()
        
        optimizer.step()
        pred=out['atom'].argmax(dim=1)
        train_correct=pred==data['atom'].y
        train_acc=int(train_correct.sum())/data['atom'].y.shape[0]
        #print(train_acc)
        model.eval()
        
        #pred=out['atom'].argmax(dim=1)
        #test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
        #test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())
        #
        train_acc_list.append(train_acc)
        #test_acc_list.append(test_acc)
        #
        #loss_list.append(loss.item())
        
        
    #test_acc_mean_list.append(np.mean(test_acc_list))
        if epcoh==0 and num ==0:
            best_score=loss
        elif best_score<loss:
            best_score=loss
            best_model = model
        loss_list.append(loss.item())
    
    print('epoch : {} , loss :{:.4f} , accuracy : {:.4f}'.format(epcoh,np.mean(loss_list),np.mean(train_acc_list)))
        
      

epoch : 0 , loss :0.6626 , accuracy : 0.6055
epoch : 1 , loss :0.6388 , accuracy : 0.6333
epoch : 2 , loss :0.6368 , accuracy : 0.6355
epoch : 3 , loss :0.6368 , accuracy : 0.6349
epoch : 4 , loss :0.6368 , accuracy : 0.6356


In [ ]:
test_score=[]
target=[]
pred_prob=[]
for num, data in enumerate(test_data):
    try:
        out=best_model(data.x_dict, data.edge_index_dict)
        pred=out['atom'].argmax(dim=1)
        test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
        test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())
        print('Test case : {} , nodes : {} , accuracy : {:.4f} '.format(num+1,data['atom'].x.shape[0],test_acc))
        target.append(data['atom'].y[data['atom'].test_mask].item())
        pred_prob.append(out['atom'][data['atom'].test_mask].detach()[0].tolist()[1])
        test_score.append(test_acc)
    except:
        pass
print('Mean accuracy : {} , AUC : {}'.format(np.mean(test_score),roc_auc_score(np.array(target),pred_prob)))

In [ ]:
plt.hist(test_score);


In [ ]:
plt.boxplot(test_score);

In [ ]:
loss_list_=[]
for i in loss_list:
    loss_list_.append(i)

In [ ]:
plt.plot(loss_list_)

In [ ]:
np.ceil(0.001)

In [ ]:
dataLoader=list(DataLoader(usersGraphDataList,batch_size=80))
avg=[]
for num, data in enumerate(dataLoader):
    ys=data["atom"].y
    unique_values, counts = torch.unique(ys, return_counts=True)
    avg.append(counts[0].item()*100/(counts[0].item()+counts[1].item()))
print( "{} , {:.2f}".format(num,np.mean(avg)))

In [ ]:
counts[0].item()

def train(data):
    model.train()
    optimizer.zero_grad()
    out=model(data.x_dict, data.edge_index_dict)
    #print(out)
    loss=criterion(out['atom'][data['atom'].train_mask],data['atom'].y[data['atom'].train_mask])
    loss.backward()
    optimizer.step()
    pred=out['atom'].argmax(dim=1)
    train_correct=pred[data['atom'].train_mask]==data['atom'].y[data['atom'].train_mask]
    train_acc=int(train_correct.sum())/int(data['atom'].train_mask.sum())
    return loss ,train_acc

def test():
    model.eval()
    out=model(data.x_dict,data.edge_index_dict)
    #print(out)
    pred=out['atom'].argmax(dim=1)
    #print(out)
    test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
    test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())

   
    classificationReport=classification_report(data['atom'].y[data['atom'].test_mask],pred[data['atom'].test_mask])
    print(classificationReport)
    return test_acc

In [ ]:
model.eval()
out=model(data.x_dict,data.edge_index_dict)
#print(out)
pred=out['atom'].argmax(dim=1)
#print(out)
test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())
classificationReport=classification_report(data['atom'].y[data['atom'].test_mask],pred[data['atom'].test_mask])
print(classificationReport)